In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions
import boto3
import sys
import re
from datetime import datetime
import configparser
from pyspark.sql.functions import col, datediff

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1670142421074_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
GoldenRecordISPINC_fact = spark.sql('''
                                        SELECT * FROM 
                                        22_customerhub.goldenrecordispinc_fact
''').repartition(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
GoldenRecordISPINC_fact.createOrReplaceTempView('GoldenRecordISPINC_fact')



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
CustomerMatchMerge_SFLTYRW_PAX = spark.sql('''
                                                SELECT * FROM 
                                                22_customerhub.customermatchmerge_sfltyrw_pax
                                                

''')

CustomerMatchMerge_SFLTYRW_PAX.createOrReplaceTempView(
    'CustomerMatchMerge_SFLTYRW_PAX')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
ContactProfileHubProcessing_fact = spark.sql('''
                                                    SELECT * FROM 
                                                    22_loyalty.contactprofilehubprocessing_fact
''')

ContactProfileHubProcessing_fact.createOrReplaceTempView(
    'ContactProfileHubProcessing_fact')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:

member_fact = spark.sql('''
                                                    SELECT * FROM 
                                                    22_loyalty.member_fact
''')

member_fact.createOrReplaceTempView('member_fact')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
points_fact1 = spark.sql('''
                                SELECT * FROM 
                                22_loyalty.points_fact
''')

#points_fact1.count()

points_fact1.createOrReplaceTempView('points_fact1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:

points_fact2 = points_fact1.na.fill(value=0, subset=["isdeleted"])
points_fact2.createOrReplaceTempView('points_fact2')
points_fact3 = points_fact2.na.fill(value=0, subset=["ishistoricalpoint"])
points_fact2.createOrReplaceTempView('points_fact3')
points_fact4 = points_fact3.na.fill(value="Credit Points",
                                    subset=["reconciliationstatus"])
points_fact = points_fact4.na.fill(value='Points', subset=["pointtypename"])
points_fact.createOrReplaceTempView('points_fact')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
memberredemption_fact = spark.sql('''
                                        SELECT * FROM 
                                        22_loyalty.memberredemption_fact
                                    ''')

memberredemption_fact.createOrReplaceTempView('memberredemption_fact')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:

tier_fact = spark.sql('''
                                        SELECT * FROM 
                                        22_loyalty.tier_fact
                                    ''')

tier_fact.createOrReplaceTempView('tier_fact')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
AncillaryRevenueChargeCodes = spark.sql('''
                                            SELECT * FROM 
                                            22_spicejetdatamart.ancillaryrevenuechargecodes

''')

AncillaryRevenueChargeCodes.createOrReplaceTempView(
    'AncillaryRevenueChargeCodes')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:

bookingpassenger_split1 = spark.sql('''
                                            SELECT * FROM 
                                            22_spicejetdatamart.bookingpassenger_split1

''').repartition(100)

bookingpassenger_split1.createOrReplaceTempView('bookingpassenger_split1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
PassengerFee = spark.sql('''
                                            SELECT * FROM 
                                            22_spicejetdatamart.passengerfee

''')

PassengerFee.createOrReplaceTempView('PassengerFee')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:

DimDate = spark.sql('''
                                            SELECT * FROM 
                                            22_spicejetdatamart.DimDate

''')

DimDate.createOrReplaceTempView('DimDate')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:

tmp_pointsissued = spark.sql('''
                                SELECT sum(pointsissued) AS pointsissued

                                     ,p.member

                                FROM points_fact p 

                                WHERE p.isdeleted  = 0 

                                     AND reconciliationstatus  = 'Credit Points'

                                     AND ishistoricalpoint  = 0

                                     AND pointtypename = 'Points'

                                --and p.member=scm.uniqueid

                                GROUP BY P.MEMBER

''')

#tmp_pointsissued.show(truncate=False)
tmp_pointsissued.createOrReplaceTempView('tmp_pointsissued')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:

tmppointsredeemed = spark.sql('''
                                    SELECT sum(pointstoredeem - pointsreverted) AS pointsredeemed

                                         ,mr.member

                                    FROM memberredemption_fact mr 

                                    WHERE mr.STATUS NOT IN (

                                              'Cancelled'

                                              ,'Transaction Reverted'

                                              )

                                    --and mr.member=scm.uniqueid

                                    GROUP BY mr.member
                                ''')

tmppointsredeemed.createOrReplaceTempView('tmppointsredeemed')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:


tmp_pointsUsed = spark.sql('''
                                SELECT SUM(mp.PointsUsed) * 100 AS pointused

                                     ,mp.member

                                FROM points_fact mp

                                --where mp.Member=scm.uniqueID

                                GROUP BY mp.member

                            ''')

tmp_pointsUsed.createOrReplaceTempView('tmp_pointsUsed')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
tmpBookingdate = spark.sql('''
                            SELECT DISTINCT cast(max(bookingdate) AS DATE) AS MAXBookingDate

                            ,org_uid

                            FROM GoldenRecordISPINC_fact

                            GROUP BY org_uid 
                            
                            ''')

tmpBookingdate.createOrReplaceTempView('tmpBookingdate')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
tmpDepartureDate = spark.sql('''

                SELECT DISTINCT cast(max(DepartureDate_PS) AS DATE) AS MAXDepartureDate, org_uid

                FROM GoldenRecordISPINC_fact g
                
                --where g.org_uid = '16127626142100316277'

                GROUP BY org_uid
                
                ''')

#tmpDepartureDate.show()

tmpDepartureDate.createOrReplaceTempView('tmpDepartureDate_PS')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
################ LastDepartureDate - START ######################

LastDepartureDate = spark.sql('''
            select DISTINCT g.ORG_UID,
         (
            SELECT Max(DepartureDate_PS)
            
            FROM GoldenRecordISPINC_fact T2 
            
            WHERE G.Org_Uid = T2.Org_Uid
            
                AND Liftstatus = 2
                
            ) LastDepartureDate
            
            FROM GoldenRecordISPINC_fact g  
            
            ''')

LastDepartureDate.createOrReplaceTempView('LastDepartureDate')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#LastDepartureDate.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
LastDepartureDate = spark.sql('''
                              SELECT DISTINCT g.ORG_UID

                              ,t1.pointsissued AS TotalPointsAccrued_12M

                              ,t8.pointsredeemed AS TotalPointsRedeemed_12M

                              ,t3.pointused AS RedemptionRatio

                              ,tg.LastDepartureDate
                                  
                                FROM GoldenRecordISPINC_fact G 
                                
                                 LEFT JOIN LastDepartureDate tg ON tg.ORG_UID = g.ORG_UID

                                 LEFT JOIN CustomerMatchMerge_SFLTYRW_PAX cl ON cl.UID = g.ORG_UID

                                 LEFT JOIN ContactProfileHubProcessing_fact m ON m.MobilePhone = g.Homephonecode --addedby amreen  ---doubt

                                 LEFT JOIN Member_fact Scm ON Scm.contact = m.uniqueid --added by amreen

                                 LEFT JOIN tier_fact t ON t.uniqueid = scm.loyaltytier --added by amreen to display tiername

                                 LEFT JOIN tmp_pointsissued t1 ON t1.member = scm.uniqueid

                                 LEFT JOIN tmppointsredeemed t8 ON t8.member = scm.uniqueid

                                 LEFT JOIN tmp_pointsUsed t3 ON t3.member = scm.uniqueid

                                 LEFT JOIN tmpBookingdate tb ON tb.org_uid = g.org_uid

                                 --WHERE g.org_uid = '16116833142100004052'

                                 GROUP BY G.ORG_UID

                                      ,cl.uid

                                      ,t1.pointsissued

                                      ,t8.pointsredeemed

                                      ,t3.pointused

                                      ,tb.MAXBookingDate
                                      
                                      ,tg.LastDepartureDate
                                      
                                      
                                      
''')

LastDepartureDate.createOrReplaceTempView('LastDepartureDate')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
################ LastDepartureDate - END ######################

#mainQuery.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
Months_12 = spark.sql('''
                SELECT g.org_uid, add_months('1900-01-01', CAST(months_between(g.MAXDepartureDate, '1900-01-01') AS INT) -12) AS Date
                            
                            FROM tmpDepartureDate_PS g
                            
                            --where g.org_uid = '16127626142100316277'
                            
            ''')

#Months_12.show()

Months_12.createOrReplaceTempView('Months_12')

Months_3 = spark.sql('''
                SELECT g.org_uid, add_months('1900-01-01', CAST(months_between(g.MAXDepartureDate, '1900-01-01') AS INT) -3) AS Date
                            
                            FROM tmpDepartureDate_PS g
                            
                            --where g.org_uid = '16127626142100316277'
                            
            ''')

Months_3.createOrReplaceTempView('Months_3')

Months_6 = spark.sql('''
                SELECT g.org_uid, add_months('1900-01-01', CAST(months_between(g.MAXDepartureDate, '1900-01-01') AS INT) -6) AS Date
                            
                            FROM tmpDepartureDate_PS g
                            
                            --where g.org_uid = '16127626142100316277'
                            
            ''')

Months_6.createOrReplaceTempView('Months_6')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
########### DomFlightFreq-START ############

DomFlightFreq_1 = spark.sql('''

    WITH CTE_1
    AS (
        SELECT DISTINCT (t2.segmentid) AS DSegmentId, t2.org_uid, t2.liftstatus, t2.International, t2.DepartureDate_PS

                FROM GoldenRecordISPINC_fact t2 
                
                 --WHERE t2.AlternateKey Between '1' AND '1000'
                  
                 
                 --t2.org_uid = '16116833142100115739'
        )
    SELECT * FROM CTE_1
    
    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
DomFlightFreq_1.createOrReplaceTempView('DomFlightFreq_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
DomFlightFreq_2 = spark.sql('''

                SELECT count(DSegmentId), org_uid 
                
                FROM DomFlightFreq_1 WHERE DSegmentId IS NOT NULL 
                
                group by Org_uid

''')

DomFlightFreq_2.createOrReplaceTempView('DomFlightFreq_2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
DomFlightFreq = spark.sql('''

    

    WITH CTE_3
    
            AS 
            
            (
                SELECT count(DSegmentId) AS DomFlightFreq, c.org_uid

                FROM DomFlightFreq_1 c

                left join tmpDepartureDate_PS td on td.org_uid = c.org_uid

                left join Months_12 m on m.org_uid = c.org_uid
                
                left join DomFlightFreq_2 c2 on c2.org_uid = c.org_uid

                WHERE --c.org_uid = '16116833142100003433'

                        --c.segmentid IS NULL
                        
                        --c.DepartureDate_PS2 = '1900-01-01'

                        --AND

                        (c.liftstatus = 2
                        
                        OR c.liftstatus IS NULL)
                        
                        AND (CAST(c.International AS VARCHAR(10)) = 'false'
                        
                        OR c.International IS NULL)

                        AND

                        (
                        
                        (c.DepartureDate_PS BETWEEN 

                                    m.date

                                    AND

                                    td.MAXDepartureDate)
                                    
                                    OR
                                    
                                    (c.DepartureDate_PS IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid
                    
                        )

            SELECT *
            FROM CTE_3
   ''')

#DomFlightFreq.count()

DomFlightFreq.createOrReplaceTempView('DomFlightFreq')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
IntFlightFreq_1 = spark.sql('''

    WITH CTE_1
    AS (
        SELECT DISTINCT (t2.segmentid) AS DSegmentId, t2.org_uid, t2.liftstatus, t2.International, t2.DepartureDate_PS

                FROM GoldenRecordISPINC_fact t2 
                
                 --WHERE t2.AlternateKey Between '1' AND '1000'
                  
                 
                 --t2.org_uid = '16116833142100003433'
        )
    SELECT * FROM CTE_1
    
    ''')

IntFlightFreq_1.createOrReplaceTempView('IntFlightFreq_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
IntFlightFreq_2 = spark.sql('''

                SELECT count(DSegmentId), org_uid 
                
                FROM IntFlightFreq_1 WHERE DSegmentId IS NOT NULL 
                
                group by Org_uid

''')

IntFlightFreq_2.createOrReplaceTempView('IntFlightFreq_2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
IntFlightFreq = spark.sql('''

    

    WITH CTE_3
    
            AS 
            
            (
                SELECT count(DSegmentId) AS IntFlightFreq, c.org_uid

                FROM DomFlightFreq_1 c

                left join tmpDepartureDate_PS td on td.org_uid = c.org_uid

                left join Months_12 m on m.org_uid = c.org_uid
                
                left join DomFlightFreq_2 c2 on c2.org_uid = c.org_uid

                WHERE --c.org_uid = '16116833142100003433'

                        --c.segmentid IS NULL
                        
                        --c.DepartureDate_PS2 = '1900-01-01'

                        --AND

                        (c.liftstatus = 2
                        
                        OR c.liftstatus IS NULL)
                        
                        AND (CAST(c.International AS VARCHAR(10)) = 'true'
                        
                        OR c.International IS NULL)

                        AND

                        (
                        
                        (c.DepartureDate_PS BETWEEN 

                                    m.date

                                    AND

                                    td.MAXDepartureDate)
                                    
                                    OR
                                    
                                    (c.DepartureDate_PS IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid
                    
                        )

            SELECT *
            FROM CTE_3 
   ''')

IntFlightFreq.createOrReplaceTempView('IntFlightFreq')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
#IntFlightFreq.count()

########## IntFlightFreq - END ###########

############# OverallFlightFreq - START #############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
OverallFlightFreq_1 = spark.sql('''
                    SELECT c.org_uid, c.DomFlightFreq, d.IntFlightFreq
                    
                    FROM DomFlightFreq c
                    
                    left join IntFlightFreq d on c.org_uid = d.org_uid
                    
                    


''')

OverallFlightFreq_1.createOrReplaceTempView('OverallFlightFreq_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
OverallFlightFreq_2 = OverallFlightFreq_1.na.fill({'IntFlightFreq': 0})

OverallFlightFreq_2.createOrReplaceTempView('OverallFlightFreq_2')

#OverallFlightFreq_2.show()

OverallFlightFreq = spark.sql('''

                SELECT org_uid, DomFlightFreq, IntFlightFreq, 
                
                (DomFlightFreq + IntFlightFreq) AS OverallFlightFreq
                
                FROM OverallFlightFreq_2

''')

OverallFlightFreq.createOrReplaceTempView('OverallFlightFreq')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
#OverallFlightFreq.count()

############# OverallFlightFreq - END #############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
############ MemberTravelCount_3M - START ##################

MemberTravelCount_3M_1 = spark.sql('''

    WITH CTE_1
    AS (
        SELECT DISTINCT (t2.segmentid) AS DSegmentId, t2.org_uid, t2.liftstatus, t2.DepartureDate_PS, t2.lineage

                FROM GoldenRecordISPINC_fact t2 
                
                 --WHERE t2.AlternateKey Between '1' AND '1000'
                  
                 --t2.org_uid = '16116833142100115739'
                             
                 --t2.org_uid = '16116833142100003433'
        )
    SELECT * FROM CTE_1
    
                    ''')

#MemberTravelCount_3M_1.show()

MemberTravelCount_3M_1.createOrReplaceTempView('MemberTravelCount_3M_1')

MemberTravelCount_3M_2 = spark.sql('''

                SELECT count(DSegmentId), org_uid 
                
                FROM MemberTravelCount_3M_1 WHERE DSegmentId IS NOT NULL 
                
                group by Org_uid

''')

MemberTravelCount_3M_2.createOrReplaceTempView('MemberTravelCount_3M_2')

MemberTravelCount_3M_3 = spark.sql('''

    

    WITH CTE_3
    
            AS 
            
            (
                SELECT count(DSegmentId) AS MemberTravelCount_3M, c.org_uid

                FROM MemberTravelCount_3M_1 c

                left join tmpDepartureDate_PS td on td.org_uid = c.org_uid

                left join Months_3 m on m.org_uid = c.org_uid
                
                left join MemberTravelCount_3M_2 c2 on c2.org_uid = c.org_uid

                WHERE --c.org_uid = '16116833142100003433'

                        --c.segmentid IS NULL
                        
                        --c.DepartureDate_PS2 = '1900-01-01'

                        --AND

                        (c.liftstatus = 2
                        
                        OR c.liftstatus IS NULL)
                        
                        AND (c.lineage  = 'SFLTY'
                        
                        OR c.lineage IS NULL)

                        AND

                        (
                        
                        (c.DepartureDate_PS BETWEEN 

                                    m.date

                                    AND

                                    td.MAXDepartureDate)
                                    
                                    OR
                                    
                                    (c.DepartureDate_PS IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid
                    
                        )

            SELECT *
            FROM CTE_3
   ''')

MemberTravelCount_3M_3.createOrReplaceTempView('MemberTravelCount_3M_3')

MemberTravelCount_3M_4 = spark.sql('''

                SELECT c.org_uid, c.DomFlightFreq, c.IntFlightFreq, c.OverallFlightFreq, d.MemberTravelCount_3M
                
                FROM OverallFlightFreq c
                
                left join MemberTravelCount_3M_3 d on c.org_uid = d.org_uid
                
                ''')

MemberTravelCount_3M_4.createOrReplaceTempView('MemberTravelCount_3M_4')

MemberTravelCount_3M = MemberTravelCount_3M_4.na.fill(
    {'MemberTravelCount_3M': 0})

MemberTravelCount_3M.createOrReplaceTempView('MemberTravelCount_3M')

#MemberTravelCount_3M.count()

########### MemberTravelCount_3M - END ##############

########### MemberTravelCount_6M - START ##############

MemberTravelCount_6M_1 = spark.sql('''

    WITH CTE_1
    AS (
        SELECT DISTINCT (t2.segmentid) AS DSegmentId, t2.org_uid, t2.liftstatus, t2.DepartureDate_PS, t2.lineage

                FROM GoldenRecordISPINC_fact t2 
                
                 --WHERE t2.AlternateKey Between '1' AND '1000'
                  
                 --t2.org_uid = '16116833142100115739'
                             
                 --t2.org_uid = '16116833142100003433'
        )
    SELECT * FROM CTE_1
    
                    ''')

#MemberTravelCount_6M_1.count()

MemberTravelCount_6M_1.createOrReplaceTempView('MemberTravelCount_6M_1')

MemberTravelCount_6M_2 = spark.sql('''

                SELECT count(DSegmentId), org_uid 
                
                FROM MemberTravelCount_6M_1 WHERE DSegmentId IS NOT NULL 
                
                group by Org_uid

''')

#MemberTravelCount_6M_2.count()

MemberTravelCount_6M_2.createOrReplaceTempView('MemberTravelCount_6M_2')

MemberTravelCount_6M_3 = spark.sql('''

    

    WITH CTE_3
    
            AS 
            
            (
                SELECT count(DSegmentId) AS MemberTravelCount_6M, c.org_uid

                FROM MemberTravelCount_6M_1 c

                left join tmpDepartureDate_PS td on td.org_uid = c.org_uid

                left join Months_6 m on m.org_uid = c.org_uid
                
                left join MemberTravelCount_6M_2 c2 on c2.org_uid = c.org_uid

                WHERE --c.org_uid = '16116833142100003433'

                        --c.segmentid IS NULL
                        
                        --c.DepartureDate_PS2 = '1900-01-01'

                        --AND

                        (c.liftstatus = 2
                        
                        OR c.liftstatus IS NULL)
                        
                        AND (c.lineage  = 'SFLTY'
                        
                        OR c.lineage IS NULL)

                        AND

                        (
                        
                        (c.DepartureDate_PS BETWEEN 

                                    m.date

                                    AND

                                    td.MAXDepartureDate)
                                    
                                    OR
                                    
                                    (c.DepartureDate_PS IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid
                    
                        )

            SELECT *
            FROM CTE_3
   ''')

MemberTravelCount_6M_3.createOrReplaceTempView('MemberTravelCount_6M_3')

MemberTravelCount_6M_4 = spark.sql('''

                SELECT c.org_uid, c.DomFlightFreq, c.IntFlightFreq, c.OverallFlightFreq, c.MemberTravelCount_3M, d.MemberTravelCount_6M
                
                FROM MemberTravelCount_3M c
                
                left join MemberTravelCount_6M_3 d on c.org_uid = d.org_uid
                
                ''')

#MemberTravelCount_6M_4.count()#createOrReplaceTempView('MemberTravelCount_6M_4')

MemberTravelCount_6M = MemberTravelCount_6M_4.na.fill(
    {'MemberTravelCount_6M': 0})

MemberTravelCount_6M.createOrReplaceTempView('MemberTravelCount_6M')

#MemberTravelCount_6M.count()

########### MemberTravelCount_6M - END ##############

########### MemberTravelCount_12M - START ##############

MemberTravelCount_12M_1 = spark.sql('''

    WITH CTE_1
    AS (
        SELECT DISTINCT (t2.segmentid) AS DSegmentId, t2.org_uid, t2.liftstatus, t2.DepartureDate_PS, t2.lineage

                FROM GoldenRecordISPINC_fact t2 
                
                 --WHERE t2.AlternateKey Between '1' AND '1000'
                  
                 --t2.org_uid = '16116833142100115739'
                             
                 --t2.org_uid = '16116833142100003433'
        )
    SELECT * FROM CTE_1
    
                    ''')

#MemberTravelCount_3M_1.show()

MemberTravelCount_12M_1.createOrReplaceTempView('MemberTravelCount_12M_1')

MemberTravelCount_12M_2 = spark.sql('''

                SELECT count(DSegmentId), org_uid 
                
                FROM MemberTravelCount_12M_1 WHERE DSegmentId IS NOT NULL 
                
                group by Org_uid

''')

MemberTravelCount_12M_2.createOrReplaceTempView('MemberTravelCount_12M_2')

MemberTravelCount_12M_3 = spark.sql('''

    

    WITH CTE_3
    
            AS 
            
            (
                SELECT count(DSegmentId) AS MemberTravelCount_12M, c.org_uid

                FROM MemberTravelCount_12M_1 c

                left join tmpDepartureDate_PS td on td.org_uid = c.org_uid

                left join Months_12 m on m.org_uid = c.org_uid
                
                left join MemberTravelCount_12M_2 c2 on c2.org_uid = c.org_uid

                WHERE --c.org_uid = '16116833142100003433'

                        --c.segmentid IS NULL
                        
                        --c.DepartureDate_PS2 = '1900-01-01'

                        --AND

                        (c.liftstatus = 2
                        
                        OR c.liftstatus IS NULL)
                        
                        AND (c.lineage  = 'SFLTY'
                        
                        OR c.lineage IS NULL)

                        AND

                        (
                        
                        (c.DepartureDate_PS BETWEEN 

                                    m.date

                                    AND

                                    td.MAXDepartureDate)
                                    
                                    OR
                                    
                                    (c.DepartureDate_PS IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid
                    
                        )

            SELECT *
            FROM CTE_3
   ''')

MemberTravelCount_12M_3.createOrReplaceTempView('MemberTravelCount_12M_3')

MemberTravelCount_12M_4 = spark.sql('''

                SELECT c.org_uid, c.DomFlightFreq, c.IntFlightFreq, c.OverallFlightFreq, c.MemberTravelCount_3M, c.MemberTravelCount_6M, d.MemberTravelCount_12M 
                
                FROM MemberTravelCount_6M c
                
                left join MemberTravelCount_12M_3 d on c.org_uid = d.org_uid
                
                ''')

MemberTravelCount_12M_4.createOrReplaceTempView('MemberTravelCount_12M_4')

MemberTravelCount_12M = MemberTravelCount_12M_4.na.fill(
    {'MemberTravelCount_12M': 0})

MemberTravelCount_12M.createOrReplaceTempView('MemberTravelCount_12M')

#MemberTravelCount_12M.count()

########### MemberTravelCount_12M - END ##############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
########## AvgDaysBkgDep - START ############

AvgDaysBkgDep = spark.sql('''

            SELECT g.ORG_UID , cast(avg(DateDif) AS INT) AvgDaysBkgDep

                     FROM (

                SELECT b.ORG_UID

                        ,segmentid

                        ,cast(bookingdate AS Date)
                        
                        ,cast(departuredate_ps AS Date)
                        
                        ,DATEDIFF(cast(departuredate_ps AS DATE), cast(bookingdate AS DATE)) DateDif

                FROM GoldenRecordISPINC_fact b
                
                --Where AlternateKey Between '1' AND '100'

                           GROUP BY b.ORG_UID

                                  ,segmentid

                                  ,bookingdate

                                  ,departuredate_ps
                                  
                                  ) aa

                     
                     
                     left join GoldenRecordISPINC_fact g
                     
                     on aa.ORG_UID = g.org_uid
                     
                     WHERE aa.ORG_UID = g.org_uid

                     GROUP BY g.ORG_UID

''')

AvgDaysBkgDep.createOrReplaceTempView('AvgDaysBkgDep')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
########## AvgDaysBkgDep - End ############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
########## WeekendBookingCount - START ############

WeekendBookingCount_1 = spark.sql('''
                SELECT g.org_uid, count(DISTINCT b.bookingid) AS WeekendBookingCount

                     FROM GoldenRecordISPINC_fact b 

                     JOIN DimDate dd ON cast(b.bookingdate AS DATE) = cast(dd.DATE AS DATE)
                     
                     left join GoldenRecordISPINC_fact g
                     
                     on b.ORG_UID = g.org_uid

                     WHERE b.org_uid = g.org_uid

                           AND IsWeekday = false
                           
                           --AND g.AlternateKey BETWEEN '1' AND '100'
                           
                    GROUP BY g.ORG_UID
                


''')#show()#count()


WeekendBookingCount_1.createOrReplaceTempView('WeekendBookingCount_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
WeekendBookingCount_2 = spark.sql('''

                    SELECT a.org_uid, a.AvgDaysBkgDep, w.WeekendBookingCount
                    
                    FROM AvgDaysBkgDep a
                    
                    left join WeekendBookingCount_1 w on w.org_uid = a.org_uid

''')

WeekendBookingCount = WeekendBookingCount_2.na.fill(
    {'WeekendBookingCount': 0})

WeekendBookingCount.createOrReplaceTempView('WeekendBookingCount')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
########## WeekendBookingCount - END ############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
########## WeekdayBookingCount - START ############

WeekdayBookingCount_1 = spark.sql('''
                SELECT g.org_uid, count(DISTINCT b.bookingid) AS WeekdayBookingCount

                     FROM GoldenRecordISPINC_fact b 

                     JOIN DimDate dd ON cast(b.bookingdate AS DATE) = cast(dd.DATE AS DATE)
                     
                     left join GoldenRecordISPINC_fact g
                     
                     on b.ORG_UID = g.org_uid

                     WHERE b.org_uid = g.org_uid

                           AND IsWeekday = true
                           
                           --AND g.AlternateKey BETWEEN '1' AND '100'
                           
                    GROUP BY g.ORG_UID
                


''')#show()#count()


WeekdayBookingCount_1.createOrReplaceTempView('WeekdayBookingCount_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
WeekdayBookingCount_2 = spark.sql('''

                    SELECT a.org_uid, a.AvgDaysBkgDep, a.WeekendBookingCount, w.WeekdayBookingCount
                    
                    FROM WeekendBookingCount a
                    
                    left join WeekdayBookingCount_1 w on w.org_uid = a.org_uid

''')

WeekdayBookingCount = WeekdayBookingCount_2.na.fill(
    {'WeekdayBookingCount': 0})

WeekdayBookingCount.createOrReplaceTempView('WeekdayBookingCount')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
########## WeekdayBookingCount - END ############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
########## AveragePAXBooking - START ###########

AveragePAXBooking_1 = spark.sql('''

                SELECT b.org_uid, count(DISTINCT b.BookingID) AS AveragePAXBooking

                     FROM GoldenRecordISPINC_fact b 

                     JOIN BookingPassenger_split1 bp ON b.BookingID = bp.BookingID

                     WHERE bp.PassengerID != b.PassengerID_BP 
                           
                     GROUP BY b.ORG_UID
                
''')

#AveragePAXBooking_1.count()#explain()

AveragePAXBooking_1.createOrReplaceTempView('AveragePAXBooking_1')

AveragePAXBooking_2 = spark.sql('''

                    SELECT a.org_uid, a.AvgDaysBkgDep, a.WeekendBookingCount, a.WeekdayBookingCount, w.AveragePAXBooking
                    
                    FROM WeekdayBookingCount a
                    
                    left join AveragePAXBooking_1 w on w.org_uid = a.org_uid

''')

AveragePAXBooking = AveragePAXBooking_2.na.fill(
    {'AveragePAXBooking': 0})

AveragePAXBooking.createOrReplaceTempView('AveragePAXBooking')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
########## AveragePAXBooking - END ###########

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
########## LastBookedDate - START ############

LastBookedDate_1 = spark.sql('''
                
                SELECT g.org_uid, max(cast(t2.bookingdate AS DATE)) AS LastBookedDate

                FROM GoldenRecordISPINC_fact t2 
                
                left join GoldenRecordISPINC_fact g

                WHERE g.ORG_UID = t2.ORG_UID
                
                --AND g.AlternateKey BETWEEN '1' AND '100'
                
                GROUP BY g.ORG_UID

''')

LastBookedDate_1.createOrReplaceTempView('LastBookedDate_1')

LastBookedDate = spark.sql('''

                    SELECT a.org_uid, a.AvgDaysBkgDep, a.WeekendBookingCount, a.WeekdayBookingCount, a.AveragePAXBooking, w.LastBookedDate
                    
                    FROM AveragePAXBooking a
                    
                    left join LastBookedDate_1 w on w.org_uid = a.org_uid

''')

LastBookedDate.createOrReplaceTempView('LastBookedDate')#show()

########## LastBookedDate - END ############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
########## UpcomingTravelDate - START #############

UpcomingTravelDate_1  = spark.sql('''
                SELECT g.org_uid,
                
                CASE

                     WHEN min(g.liftstatus) = 0

                           AND max(cast(g.DepartureDate_PS AS DATE)) > cast(current_date() AS DATE)

                           THEN max(g.DepartureDate_PS)

                     ELSE NULL

                     END UpcomingTravelDate
                
                FROM GoldenRecordISPINC_fact t2 
                
                left join GoldenRecordISPINC_fact g
                
                
                
                WHERE g.ORG_UID = t2.ORG_UID
                
                GROUP BY g.ORG_UID

''')

UpcomingTravelDate_1.createOrReplaceTempView('UpcomingTravelDate_1')

UpcomingTravelDate = spark.sql('''

                    SELECT a.org_uid, a.AvgDaysBkgDep, a.WeekendBookingCount, a.WeekdayBookingCount, a.AveragePAXBooking, 
                    
                    a.LastBookedDate, w.UpcomingTravelDate
                    
                    FROM LastBookedDate a
                    
                    left join UpcomingTravelDate_1 w on w.org_uid = a.org_uid

''')

UpcomingTravelDate.createOrReplaceTempView('UpcomingTravelDate')#show()

########## UpcomingTravelDate - END #############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
########### DomFlightSpend_NAV_12M - START ###############

Months_12_BD = spark.sql('''
                SELECT g.org_uid, add_months('1900-01-01', CAST(months_between(g.MAXBookingDate, '1900-01-01') AS INT) -12) AS Date
                            
                            FROM tmpBookingDate g
                            
                            --where g.org_uid = '16127626142100316277'
                            
            ''')

Months_12_BD.createOrReplaceTempView('Months_12_BD')

########## DomFlightSpend_NAV_12M ##########

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
DomFlightSpend_NAV_12M = spark.sql('''
                
            SELECT sum(DiscountedBasefare) AS DomFlightSpend_NAV_12M, c.org_uid

                FROM GoldenRecordISPINC_fact c

                left join tmpBookingdate td on td.org_uid = c.org_uid

                left join Months_12_BD m on m.org_uid = c.org_uid

                WHERE
                
                       --c.org_uid = '16121934962100992431'
                       
                       --AND
                            
                       (CAST(c.International AS VARCHAR(10)) = 'false'
                        
                        OR c.International IS NULL)

                        AND
                        
                        (c.lineage = 'NAV' OR c.lineage IS NULL)
                        
                        AND

                        (
                        
                        (c.BookingDate BETWEEN 

                                    m.date

                                    AND

                                    td.MAXBookingDate)
                                    
                                    OR
                                    
                                    (c.BookingDate IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid


''')#.show(50)

#DomFlightSpend_NAV_12M.count()

DomFlightSpend_NAV_12M.createOrReplaceTempView('DomFlightSpend_NAV_12M')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
########### DomFlightSpend_NAV_12M - END ###############

########### DomFlightSpend_lty_12M - START ###############

DomFlightSpend_lty_12M_1 = spark.sql('''
                
            SELECT sum(DiscountedBasefare) AS DomFlightSpend_lty_12M, c.org_uid

                FROM GoldenRecordISPINC_fact c

                left join tmpBookingdate td on td.org_uid = c.org_uid

                left join Months_12_BD m on m.org_uid = c.org_uid

                WHERE
                
                       
                       --c.org_uid = '16121934962100992431'
                       --AND
                       
                            
                       (CAST(c.International AS VARCHAR(10)) = 'false'
                        
                        OR c.International IS NULL)

                        AND
                        
                        (c.lineage = 'SFLTY' OR c.lineage IS NULL)
                        
                        AND

                        (
                        
                        (c.BookingDate BETWEEN 

                                    m.date

                                    AND

                                    td.MAXBookingDate)
                                    
                                    OR
                                    
                                    (c.BookingDate IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid


''')#.show(50)

DomFlightSpend_lty_12M_1.createOrReplaceTempView('DomFlightSpend_lty_12M_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
#DomFlightSpend_lty_12M_1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
DomFlightSpend_lty_12M = spark.sql('''
                                        SELECT a.org_uid, a.DomFlightSpend_NAV_12M, w.DomFlightSpend_lty_12M
                                        
                                        FROM DomFlightSpend_NAV_12M a 
                                        
                                        left join DomFlightSpend_lty_12M_1 w on w.org_uid = a.org_uid

''')

DomFlightSpend_lty_12M.createOrReplaceTempView('DomFlightSpend_lty_12M')#show(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
DomFlightSpend_lty_12M.show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------------+----------------------+
|org_uid             |DomFlightSpend_NAV_12M|DomFlightSpend_lty_12M|
+--------------------+----------------------+----------------------+
|16116833142100001901|0.00                  |null                  |
|16116833142100003584|0.00                  |null                  |
|16116833142100004719|0.00                  |null                  |
|16116833142100006328|3950.00               |null                  |
|16116833142100009495|0.00                  |null                  |
|16116833142100010128|3296.00               |null                  |
|16116833142100012404|4713.00               |null                  |
|16116833142100012639|4589.00               |null                  |
|16116833142100013587|2589.00               |null                  |
|16116833142100013744|12955.00              |null                  |
|16116833142100013990|9843.00               |null                  |
|16116833142100022668|13427.00    

In [73]:
########### DomFlightSpend_lty_12M - END ###############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
########### IntFlightSpend_NAV_12M - START ###############

IntFlightSpend_NAV_12M_1 = spark.sql('''
                
            SELECT sum(DiscountedBasefare) AS IntFlightSpend_NAV_12M, c.org_uid

                FROM GoldenRecordISPINC_fact c

                left join tmpBookingdate td on td.org_uid = c.org_uid

                left join Months_12_BD m on m.org_uid = c.org_uid

                WHERE
                
                       
                       
                       --c.org_uid = '16116833142100000128'
                       
                       
                            
                       (CAST(c.International AS VARCHAR(10)) = 'true'
                        
                        OR c.International IS NULL)

                        AND
                        
                        (c.lineage = 'NAV' OR c.lineage IS NULL)
                        
                        AND

                        (
                        
                        (c.BookingDate BETWEEN 

                                    m.date

                                    AND

                                    td.MAXBookingDate)
                                    
                                    OR
                                    
                                    (c.BookingDate IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid


''')#.show(50)

#IntFlightSpend_NAV_12M_1.count()

#IntFlightSpend_NAV_12M_2= IntFlightSpend_NAV_12M_1.na.fill(
    #{'IntFlightSpend_NAV_12M': 0.00})

#IntFlightSpend_NAV_12M_2.show(50)

IntFlightSpend_NAV_12M_1.createOrReplaceTempView('IntFlightSpend_NAV_12M_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
IntFlightSpend_NAV_12M = spark.sql('''

                                    SELECT a.*, w.IntFlightSpend_NAV_12M
                                        
                                        FROM DomFlightSpend_lty_12M a 
                                        
                                        left join IntFlightSpend_NAV_12M_1 w on w.org_uid = a.org_uid

''')

IntFlightSpend_NAV_12M.createOrReplaceTempView('IntFlightSpend_NAV_12M')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
#IntFlightSpend_NAV_12M.show()

########### IntFlightSpend_NAV_12M - END ###############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
########### IntFlightSpend_lty_12M - START ###############

IntFlightSpend_lty_12M_1 = spark.sql('''
                
            SELECT sum(DiscountedBasefare) AS IntFlightSpend_lty_12M, c.org_uid

                FROM GoldenRecordISPINC_fact c

                left join tmpBookingdate td on td.org_uid = c.org_uid

                left join Months_12_BD m on m.org_uid = c.org_uid

                WHERE
                
                      
                       
                       
                            
                       (CAST(c.International AS VARCHAR(10)) = 'true'
                        
                        OR c.International IS NULL)

                        AND
                        
                        (c.lineage = 'SFLTY' OR c.lineage IS NULL)
                        
                        AND

                        (
                        
                        (c.BookingDate BETWEEN 

                                    m.date

                                    AND

                                    td.MAXBookingDate)
                                    
                                    OR
                                    
                                    (c.BookingDate IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid


''')#.show(50)

#IntFlightSpend_lty_12M_2= IntFlightSpend_lty_12M_1.na.fill(
    #{'IntFlightSpend_lty_12M': 0.00})

IntFlightSpend_lty_12M_1.createOrReplaceTempView('IntFlightSpend_lty_12M_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
IntFlightSpend_lty_12M = spark.sql('''
                                        SELECT a.*, w.IntFlightSpend_lty_12M
                                        
                                        FROM IntFlightSpend_NAV_12M a 
                                        
                                        left join IntFlightSpend_lty_12M_1 w on w.org_uid = a.org_uid

''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
IntFlightSpend_lty_12M.show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------------+----------------------+----------------------+----------------------+
|org_uid             |DomFlightSpend_NAV_12M|DomFlightSpend_lty_12M|IntFlightSpend_NAV_12M|IntFlightSpend_lty_12M|
+--------------------+----------------------+----------------------+----------------------+----------------------+
|16116833142100001901|0.00                  |null                  |null                  |null                  |
|16116833142100003584|0.00                  |null                  |null                  |null                  |
|16116833142100004719|0.00                  |null                  |null                  |null                  |
|16116833142100006328|3950.00               |null                  |null                  |null                  |
|16116833142100009495|0.00                  |null                  |0.00                  |null                  |
|16116833142100010128|3296.00               |null                  |null        

In [66]:
########### IntFlightSpend_lty_12M - END ###############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#IntFlightNavLty = IntFlightSpend_lty_12M.na.fill({'DomFlightSpend_lty_12M': 0.00}) \
#.na.fill({'IntFlightSpend_lty_12M': 0.00}).na.fill({'IntFlightSpend_NAV_12M': 0.00}) 

In [79]:
IntFlightSpend_lty_12M.createOrReplaceTempView('IntFlightSpend_lty_12M')#show(50)

########## Both INTFLIGHTSPEND OF NAV & LTY ############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
#IntFlightNavLty.createOrReplaceTempView('IntFlightNavLty')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
#IntFlightNavLty.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
########## OverallFlightSpend_12 - START ##########

OverallFlightSpend_12 = spark.sql('''
                                            SELECT 
                                            
                                            g.org_uid,
                                            
                                            g.DomFlightSpend_NAV_12M,
                                            
                                            g.DomFlightSpend_lty_12M,
                                            
                                            g.IntFlightSpend_NAV_12M,
                                            
                                            g.IntFlightSpend_lty_12M
                                            
                                            ,(g.DomFlightSpend_NAV_12M + g.IntFlightSpend_NAV_12M) AS OverallFlightSpend_12_NAV
                                            
                                            ,(g.DomFlightSpend_lty_12M + g.IntFlightSpend_lty_12M) AS OverallFlightSpend_12_LTY
                                            
                                            FROM IntFlightSpend_lty_12M g

''')

OverallFlightSpend_12.createOrReplaceTempView('OverallFlightSpend_12')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
OverallFlightSpend_12.show(20,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------------+----------------------+----------------------+----------------------+-------------------------+-------------------------+
|org_uid             |DomFlightSpend_NAV_12M|DomFlightSpend_lty_12M|IntFlightSpend_NAV_12M|IntFlightSpend_lty_12M|OverallFlightSpend_12_NAV|OverallFlightSpend_12_LTY|
+--------------------+----------------------+----------------------+----------------------+----------------------+-------------------------+-------------------------+
|16116833142100001901|0.00                  |null                  |null                  |null                  |null                     |null                     |
|16116833142100003584|0.00                  |null                  |null                  |null                  |null                     |null                     |
|16116833142100004719|0.00                  |null                  |null                  |null                  |null                     |null                     

In [84]:
########## OverallFlightSpend_12 - END ##########

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
########### TicketValueWebApp ##############

TicketValueWebApp_DOM_1 = spark.sql('''

            
            SELECT sum(DiscountedBasefare) AS TicketValueWebApp_DOM, c.org_uid

                FROM GoldenRecordISPINC_fact c

                left join tmpBookingdate td on td.org_uid = c.org_uid

                left join Months_12_BD m on m.org_uid = c.org_uid

                WHERE
                
                       
                            
                       (CAST(c.International AS VARCHAR(10)) = 'false'
                        
                        OR c.International IS NULL)

                        AND
                        
                        (c.ChannelType = 2 OR c.ChannelType IS NULL)
                        
                        AND

                        (
                        
                        (c.BookingDate BETWEEN 

                                    m.date

                                    AND

                                    td.MAXBookingDate)
                                    
                                    OR
                                    
                                    (c.BookingDate IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid
                    
                    
                    
                    


''')

#TicketValueWebApp_DOM_1.count()

TicketValueWebApp_DOM = TicketValueWebApp_DOM_1.na.fill({'TicketValueWebApp_DOM': 0.00})

TicketValueWebApp_DOM.createOrReplaceTempView('TicketValueWebApp_DOM')

TicketValueWebApp_INT_1 = spark.sql('''

            
            SELECT sum(DiscountedBasefare) AS TicketValueWebApp_INT, c.org_uid

                FROM GoldenRecordISPINC_fact c

                left join tmpBookingdate td on td.org_uid = c.org_uid

                left join Months_12_BD m on m.org_uid = c.org_uid

                WHERE
                
                       
                            
                       (CAST(c.International AS VARCHAR(10)) = 'true'
                        
                        OR c.International IS NULL)

                        AND
                        
                        (c.ChannelType = 2 OR c.ChannelType IS NULL)
                        
                        AND

                        (
                        
                        (c.BookingDate BETWEEN 

                                    m.date

                                    AND

                                    td.MAXBookingDate)
                                    
                                    OR
                                    
                                    (c.BookingDate IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid
                    
                    
                    
                    


''')

#TicketValueWebApp_INT_1.count()

TicketValueWebApp_INT = TicketValueWebApp_INT_1.na.fill({'TicketValueWebApp_INT': 0.00})

#TicketValueWebApp_INT.show()

TicketValueWebApp_INT.createOrReplaceTempView('TicketValueWebApp_INT')

TicketValueWebApp_1 = spark.sql('''

            SELECT c.org_uid, c.TicketValueWebApp_DOM, w.TicketValueWebApp_INT
            
            FROM TicketValueWebApp_DOM c
            
            LEFT JOIN TicketValueWebApp_INT w on c.org_uid = w.org_uid


''')

#TicketValueWebApp_1.show()

TicketValueWebApp_2 = TicketValueWebApp_1.na.fill({'TicketValueWebApp_DOM': 0.00}) \
.na.fill({'TicketValueWebApp_INT': 0.00})

TicketValueWebApp_2.createOrReplaceTempView('TicketValueWebApp_2')

TicketValueWebApp_3 = spark.sql('''

          SELECT a.*, (TicketValueWebApp_DOM + TicketValueWebApp_INT) AS TicketValueWebApp
          
          FROM TicketValueWebApp_2 a
          
          ''')

TicketValueWebApp_3.createOrReplaceTempView('TicketValueWebApp_3')

TicketValueWebApp = spark.sql('''
                                    SELECT a.*, w.TicketValueWebApp 
                                    
                                    FROM OverallFlightSpend_12 a
                                    
                                    LEFT JOIN TicketValueWebApp_3 w on w.org_uid = a.org_uid


''')

#TicketValueWebApp.count()

TicketValueWebApp.createOrReplaceTempView('TicketValueWebApp')

########### TicketValueWebApp ##############

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
############ AncilliarySpend_NAV_12M ##############

AncilliarySpend_NAV_12M = spark.sql('''

                SELECT c.org_uid, sum(ancillaryamount) AS AncilliarySpend_NAV_12M 

                FROM GoldenRecordISPINC_fact c

                left join tmpBookingdate td on td.org_uid = c.org_uid

                left join Months_12_BD m on m.org_uid = c.org_uid

                WHERE              
                                             
                        (c.lineage = 'NAV' OR c.lineage IS NULL)
                        
                        AND

                        (
                        
                        (c.BookingDate BETWEEN 

                                    m.date

                                    AND

                                    td.MAXBookingDate)
                                    
                                    OR
                                    
                                    (c.BookingDate IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid


''')

#AncilliarySpend_NAV_12M.count()

AncilliarySpend_NAV_12M.createOrReplaceTempView('AncilliarySpend_NAV_12M')

In [ ]:
############ AncilliarySpend_NAV_12M ##############

In [ ]:
############ AncilliarySpend_Lty_12M ##############

AncilliarySpend_Lty_12M = spark.sql('''
                SELECT c.org_uid, sum(ancillaryamount) AS AncilliarySpend_Lty_12M 

                FROM GoldenRecordISPINC_fact c

                left join tmpBookingdate td on td.org_uid = c.org_uid

                left join Months_12_BD m on m.org_uid = c.org_uid

                WHERE
                
                    --c.org_uid IN ('16116833142100004052','16116833142100003433')
                
                       
                        
                        (c.lineage = 'SFLTY' OR c.lineage IS NULL)
                        
                        AND

                        (
                        
                        (c.BookingDate BETWEEN 

                                    m.date

                                    AND

                                    td.MAXBookingDate)
                                    
                                    OR
                                    
                                    (c.BookingDate IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid


''')

#AncilliarySpend_Lty_12M.count()

AncilliarySpend_Lty_12M.createOrReplaceTempView('AncilliarySpend_Lty_12M')

############ AncilliarySpend_Lty_12M ##############

In [ ]:
############ AncilliarySpend ##############

AncilliarySpend_1 = spark.sql('''
                                SELECT a.org_uid, a.AncilliarySpend_NAV_12M, w.AncilliarySpend_Lty_12M
                                
                                FROM AncilliarySpend_NAV_12M a
                                
                                LEFT JOIN AncilliarySpend_Lty_12M w on a.org_uid = w.org_uid
                                
                                


''')

AncilliarySpend_1.createOrReplaceTempView('AncilliarySpend_1')

#AncilliarySpend_2 = AncilliarySpend_1.na.fill({'AncilliarySpend_NAV_12M': 0.00}) \
#.na.fill({'AncilliarySpend_Lty_12M': 0.00})

#AncilliarySpend_2.createOrReplaceTempView('AncilliarySpend_2')

AncilliarySpend = spark.sql('''
                    SELECT a.*, b.AncilliarySpend_NAV_12M, b.AncilliarySpend_Lty_12M
                    
                    FROM TicketValueWebApp a
                    
                    LEFT JOIN AncilliarySpend_1 b on b.org_uid = a.org_uid
                    
                    ''')

AncilliarySpend.createOrReplaceTempView('AncilliarySpend') 

In [ ]:
#AncilliarySpend.show()

In [ ]:
############### OverallFlightSpend_12 ###############

OverallFlightSpend_12 = spark.sql('''

                
                SELECT a.*,
                
                (a.DomFlightSpend_NAV_12M + a.IntFlightSpend_NAV_12M + a.AncilliarySpend_NAV_12M) AS OverallAirtravelSpend_NAV_12M,
                
                (a.DomFlightSpend_Lty_12M + a.IntFlightSpend_Lty_12M + a.AncilliarySpend_Lty_12M) AS OverallAirtravelSpend_Lty_12M
                
                FROM AncilliarySpend a
                
                LEFT JOIN AncilliarySpend_1 b on a.org_uid = b.org_uid


''')

OverallFlightSpend_12.createOrReplaceTempView('OverallFlightSpend_12')

In [ ]:
############### OverallFlightSpend_12 - END ###############

############### AncillaryPurchaseCount - END ###############

In [ ]:
AncillaryPurchaseCount_NAV_12 = spark.sql('''
            SELECT org_uid, count(Counts) AS AncillaryPurchaseCount_NAV_12M
            
            FROM 
            (
            
            SELECT c.org_uid, count(*) AS Counts

                FROM GoldenRecordISPINC_fact c
                
                left join tmpBookingdate td on td.org_uid = c.org_uid

                left join Months_12_BD m on m.org_uid = c.org_uid

                join PassengerFee pf on pf.PassengerID = c.PassengerID_BP

                join AncillaryRevenueChargeCodes a on pf.FeeCode = a.Chargecode
                
                AND AncillaryRevenueFlag = 1

                WHERE
                
                       
                        
                        (c.lineage = 'NAV' OR c.lineage IS NULL)
                        
                        AND

                        (
                        
                        (c.BookingDate BETWEEN 

                                    m.date

                                    AND

                                    td.MAXBookingDate)
                                    
                                    OR
                                    
                                    (c.BookingDate IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid
                    
                                ,PassengerID

                                  ,FeeNumber

                                  ,FeeCode
                                  
                                  ) aa
                                  
                                 
                                  
                                 GROUP BY 
                                  
                                 org_uid
                                  
                                  
                                  
                                  



''')

#AncillaryPurchaseCount_NAV_12.count()

AncillaryPurchaseCount_NAV_12.createOrReplaceTempView('AncillaryPurchaseCount_NAV_12')

In [ ]:
AncillaryPurchaseCount_Lty_12 = spark.sql('''
            SELECT org_uid, count(Counts) AS AncillaryPurchaseCount_Lty_12M
            
            FROM 
            (
            
            SELECT c.org_uid, count(*) AS Counts

                FROM GoldenRecordISPINC_fact c
                
                left join tmpBookingdate td on td.org_uid = c.org_uid

                left join Months_12_BD m on m.org_uid = c.org_uid

                join PassengerFee pf on pf.PassengerID = c.PassengerID_BP

                join AncillaryRevenueChargeCodes a on pf.FeeCode = a.Chargecode
                
                AND AncillaryRevenueFlag = 1

                WHERE
                
                       
                        
                        (c.lineage = 'SFLTY' OR c.lineage IS NULL)
                        
                        AND

                        (
                        
                        (c.BookingDate BETWEEN 

                                    m.date

                                    AND

                                    td.MAXBookingDate)
                                    
                                    OR
                                    
                                    (c.BookingDate IS NULL)
                                    
                                    )

                    GROUP BY c.org_uid
                    
                                ,PassengerID

                                  ,FeeNumber

                                  ,FeeCode
                                  
                                  ) aa
                                  
                                  GROUP BY 
                                  
                                 org_uid
                                 
                                 ''')
                                  
                                 
                                  
                          

In [ ]:
AncillaryPurchaseCount_Lty_12.createOrReplaceTempView('AncillaryPurchaseCount_Lty_12')

AncillaryPurchaseCount = spark.sql('''

                SELECT a.*, b.AncillaryPurchaseCount_NAV_12M, c.AncillaryPurchaseCount_Lty_12M
                
                FROM OverallFlightSpend_12 a 
                
                LEFT JOIN AncillaryPurchaseCount_NAV_12 b on b.org_uid = a.org_uid
                
                LEFT JOIN AncillaryPurchaseCount_Lty_12 c on c.org_uid = a.org_uid


''')

AncillaryPurchaseCount.createOrReplaceTempView('AncillaryPurchaseCount')